In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate=3e-4
eval_iters =250
eval_iters = 250

cpu


In [2]:
with open ("/Users/sebascordova/Desktop/Code proyects/wizard_of_oz.txt", 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [3]:

torch_1 = torch.rand(100, 100, 100, 100, device=device)
torch_2 = torch.rand(100, 100, 100, 100, device=device)


start_time = time.time()

result = torch.matmul(torch_1, torch_2)
end_time = time.time()
duration = end_time - start_time

print(f"Duración: {duration} segundos")

Duración: 2.273132085800171 segundos


In [4]:

# Mapeo de los carácteres a un indice y su posterior mapeo para cada indice entero de vuelta a su carácter correspondiente.
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

#Codificación y decodificación
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    contexto = x[:t+1]
    tarjet = y[t]
    print('When input is', contexto, ' tarjet is ', tarjet)

When input is tensor([91])  tarjet is  tensor(48)
When input is tensor([91, 48])  tarjet is  tensor(65)
When input is tensor([91, 48, 65])  tarjet is  tensor(62)
When input is tensor([91, 48, 65, 62])  tarjet is  tensor(1)
When input is tensor([91, 48, 65, 62,  1])  tarjet is  tensor(44)
When input is tensor([91, 48, 65, 62,  1, 44])  tarjet is  tensor(75)
When input is tensor([91, 48, 65, 62,  1, 44, 75])  tarjet is  tensor(72)
When input is tensor([91, 48, 65, 62,  1, 44, 75, 72])  tarjet is  tensor(67)


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):

    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+batch_size +1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([ 37123, 179730, 110778, 161216])
inputs:
tensor([[ 1, 63, 69, 58, 73, 73, 62, 61],
        [77, 65, 62, 82,  1, 59, 62, 60],
        [66, 69, 66, 71, 64, 69, 82,  1],
        [76, 77, 66, 69, 69,  1, 76, 77]])
targets:
tensor([[63, 69, 58, 73],
        [65, 62, 82,  1],
        [69, 66, 71, 64],
        [77, 66, 69, 69]])


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
index

NameError: name 'index' is not defined

In [17]:
nn.Embedding(vocab_size, vocab_size)

Embedding(92, 92)

In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        print(logits)
        print(logits.shape)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


tensor([[[ 0.1654, -1.9144,  0.7914, -1.7639,  1.3786,  0.2088,  0.8032,
           1.1583,  1.2041, -0.7040,  0.1795, -0.7888,  0.2158, -0.4319,
          -0.1173,  0.0167, -0.4726,  0.5519, -1.3025,  0.6651,  0.8353,
          -0.1363, -0.8086,  0.6246, -0.3666,  0.0897,  0.2697, -0.1624,
           2.3895,  1.7811,  0.2976, -1.0460, -0.1140, -1.5229, -0.5100,
          -0.5634,  1.4665, -1.1670, -0.3774, -1.3214,  0.3039, -0.0794,
           0.0360, -1.1917, -0.1140, -1.0708, -1.3754, -0.7742,  0.3648,
          -0.4169, -1.1924, -1.7961, -0.1747, -0.7036,  0.0392,  1.0782,
           0.5717,  0.0385, -0.1126, -0.4802, -0.5747, -1.4965, -0.4230,
           0.1297, -0.6021,  2.3944, -1.8851,  1.4513, -0.7473,  0.3723,
          -1.5005,  0.0333,  0.1126,  1.0744, -0.3035,  1.7009,  0.9935,
           0.3691,  0.3458,  0.2546,  0.9280,  0.3316,  0.7888,  0.5670,
           0.6685,  1.2263, -0.6795, -0.7071,  0.2987,  1.1725,  0.4740,
          -1.0748]]], grad_fn=<EmbeddingBackward0>)

In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([102428, 179570,  80278,  68700])


RuntimeError: shape '[32]' is invalid for input of size 16

In [11]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


n/D %-p[fbF;t'yDNU84L:FI'PA’c-"OI)
D •N tNzwT3‘﻿Pt?J&BKb(6—-•wqE]HypH"xpU™6O™_67uQ0UK,XRyEMl’!?qe—*y5/8.iS•m53Eg*6I!vFGrZFtG(9H/n•hX*Zf8Rg1cH”c•G?K™Z QtNM_“xb0BqC.!KhL?O4o“KQ0[,h"Aj[8”KA/.!eX—-V2X.Vxm:Bq1R'M  lh!]#H(•qlt#uG(d’!‘”4
3KAjAw,:Y•Hkm’4DbI*]GIR!21Rg3?'-k3’Mlrz—/1Pf*3hr#IiLu4-T3?8.i™H‘﻿(bqu[;:—,8”I﻿y"wbD0l’Zz-i]y1wg1v#(dGq/Ei7z*vy9VK4CNT53FBc1A7)w™i[Sxd(dn*™IZFX?P9!1ABgCbu[GHNPAgSS﻿Je:D uI";E8SXa—[oHglgX)5cV-"e;%S)q8(Xn•Y“Ou&zL?0 "AlQ0tU#1uLtVCLU'a-OzOO•”CBTn*dQPhnHE*Jj/a)L™EwA_8"—hqB5"
